In [7]:
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping
import random
import numpy as np

In [2]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [ ]:
# Normalize pixel values to the range [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [3]:
# Convert labels to categorical one-hot encoding
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [4]:
# Split the training dataset into validation and partial training sets
val_images = X_train[:10000]
partial_images = X_train[10000:]

val_labels = y_train[:10000]
partial_labels = y_train[10000:]

In [6]:
def create_custom_cnn_model(f1, f2, f3, kernel_size, activation_1, activation_2, dropout_1, dropout_2, optimizer, epochs):
    custom_model = models.Sequential()
    custom_model.add(layers.Conv2D(filters=f1, kernel_size=kernel_size, activation=activation_1, input_shape=(32, 32, 3)))
    custom_model.add(layers.Conv2D(filters=f1, kernel_size=kernel_size, activation=activation_1))
    custom_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    custom_model.add(layers.Conv2D(filters=f2, kernel_size=kernel_size, activation=activation_2))
    custom_model.add(layers.Conv2D(filters=f2, kernel_size=kernel_size, activation=activation_2))
    custom_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    custom_model.add(layers.Flatten())
    custom_model.add(layers.Dropout(rate=dropout_1))
    custom_model.add(layers.Dense(units=f3, activation=activation_2))
    custom_model.add(layers.Dropout(rate=dropout_2))
    custom_model.add(layers.Dense(units=10, activation="softmax"))

    custom_model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    early_stopping = EarlyStopping(monitor="val_accuracy", patience=7)
    custom_model.fit(partial_images, partial_labels, validation_data=(val_images, val_labels), epochs=epochs, batch_size=100, callbacks=[early_stopping], verbose=0)

    return custom_model

In [8]:
def initialize_parameters():
    parameters = {}
    parameters["filters_1"] = random.choice([32, 64])
    parameters["filters_2"] = random.choice([64, 128])
    parameters["filters_3"] = random.choice([128, 256, 512])
    parameters["kernel_size"] = random.choice([(3, 3), (5, 5)])
    parameters["activation_1"] = random.choice(["relu", "sigmoid", "tanh"])
    parameters["activation_2"] = random.choice(["relu", "sigmoid", "tanh"])
    parameters["dropout_1"] = round(random.uniform(0.1, 0.5), 1)
    parameters["dropout_2"] = round(random.uniform(0.1, 0.5), 1)
    parameters["optimizer"] = random.choice(["adam", "adagrad", "SGD", "rmsprop"])
    parameters["epochs"] = np.random.randint(50, 100)
    return parameters

In [9]:
def generate_population(n):
    population = []
    for _ in range(n):
        chromosome = initialize_parameters()
        population.append(chromosome)
    return population

In [11]:
def evaluate_fitness(model):
    evaluation_metrics = model.evaluate(X_test, y_test)
    accuracy = evaluation_metrics[1]
    return accuracy

In [12]:
def selection(population_fitness):
    total = sum(population_fitness)
    percentage = [round((x / total) * 100) for x in population_fitness]
    selection_wheel = []
    for pop_index, num in enumerate(percentage):
        selection_wheel.extend([pop_index] * num)
    parent1_ind = random.choice(selection_wheel)
    parent2_ind = random.choice(selection_wheel)
    return [parent1_ind, parent2_ind]

In [13]:
def crossover(parent1, parent2):
    child1 = {}
    child2 = {}

    child1["filters_1"] = random.choice([parent1["filters_1"], parent2["filters_1"]])
    child1["filters_2"] = random.choice([parent1["filters_2"], parent2["filters_2"]])
    child1["filters_3"] = random.choice([parent1["filters_3"], parent2["filters_3"]])

    child2["filters_1"] = random.choice([parent1["filters_1"], parent2["filters_1"]])
    child2["filters_2"] = random.choice([parent1["filters_2"], parent2["filters_2"]])
    child2["filters_3"] = random.choice([parent1["filters_3"], parent2["filters_3"]])

    child1["kernel_size"] = random.choice([parent1["kernel_size"], parent2["kernel_size"]])
    child2["kernel_size"] = random.choice([parent1["kernel_size"], parent2["kernel_size"]])

    child1["activation_1"] = parent1["activation_2"]
    child2["activation_1"] = parent2["activation_2"]

    child1["activation_2"] = parent2["activation_1"]
    child2["activation_2"] = parent1["activation_1"]

    child1["dropout_1"] = parent1["dropout_1"]
    child2["dropout_1"] = parent2["dropout_1"]

    child1["dropout_2"] = parent2["dropout_2"]
    child2["dropout_2"] = parent1["dropout_2"]

    child1["optimizer"] = parent2["optimizer"]
    child2["optimizer"] = parent1["optimizer"]

    child1["epochs"] = parent1["epochs"]
    child2["epochs"] = parent2["epochs"]

    return [child1, child2]

In [14]:
def mutation(chromosome):
    flag = random.randint(0, 40)
    if flag <= 20:
        chromosome["epochs"] += random.randint(0, 10)
    return chromosome

In [15]:
generations = 3
threshold = 90
num_pop = 10

In [ ]:
population = generate_population(num_pop)

for generation in range(generations):
    population_fitness = []
    for chromosome in population:
        filters_1 = chromosome["filters_1"]
        filters_2 = chromosome["filters_2"]
        filters_3 = chromosome["filters_3"]
        kernel_size = chromosome["kernel_size"]
        activation_1 = chromosome["activation_1"]
        activation_2 = chromosome["activation_2"]
        dropout_1 = chromosome["dropout_1"]
        dropout_2 = chromosome["dropout_2"]
        optimizer = chromosome["optimizer"]
        epochs = chromosome["epochs"]

        try:
            model = create_custom_cnn_model(filters_1, filters_2, filters_3, kernel_size, activation_1, activation_2, dropout_1, dropout_2, optimizer, epochs)
            acc = evaluate_fitness(model)
            print("Parameters: ", chromosome)
            print("Accuracy: ", round(acc, 3))
        except:
            acc = 0
            print("Parameters: ", chromosome)
            print("Invalid parameters - Build fail")

        population_fitness.append(acc)

    parents_ind = selection(population_fitness)
    parent1 = population[parents_ind[0]]
    parent2 = population[parents_ind[1]]

    children = crossover(parent1, parent2)
    child1 = mutation(children[0])
    child2 = mutation(children[1])

    population.append(child1)
    population.append(child2)

    print("Generation ", generation + 1, " Outcome: ")

    if max(population_fitness) >= threshold:
        print("Obtained desired accuracy: ", max(population_fitness))
        break
    else:
        print("Maximum accuracy in generation {}: {}".format(generation + 1, max(population_fitness)))

    first_min = min(population_fitness)
    first_min_ind = population_fitness.index(first_min)
    population.remove(population[first_min_ind])
    second_min = min(population_fitness)
    second_min_ind = population_fitness.index(second_min)
    population.remove(population[second_min_ind])

313/313 [==============================] - 1s 4ms/step - loss: 1.3515 - accuracy: 0.5037
Parameters:  {'filters_1': 32, 'filters_2': 64, 'filters_3': 128, 'kernel_size': (5, 5), 'activation_1': 'sigmoid', 'activation_2': 'relu', 'dropout_1': 0.2, 'dropout_2': 0.2, 'optimizer': 'adagrad', 'epochs': 93}
Accuracy:  0.504
313/313 [==============================] - 1s 3ms/step - loss: 0.8867 - accuracy: 0.6993
Parameters:  {'filters_1': 32, 'filters_2': 64, 'filters_3': 256, 'kernel_size': (3, 3), 'activation_1': 'relu', 'activation_2': 'tanh', 'dropout_1': 0.1, 'dropout_2': 0.1, 'optimizer': 'SGD', 'epochs': 59}
Accuracy:  0.699
313/313 [==============================] - 1s 3ms/step - loss: 1.5505 - accuracy: 0.4277
Parameters:  {'filters_1': 64, 'filters_2': 64, 'filters_3': 512, 'kernel_size': (3, 3), 'activation_1': 'tanh', 'activation_2': 'sigmoid', 'dropout_1': 0.2, 'dropout_2': 0.1, 'optimizer': 'adagrad', 'epochs': 65}
Accuracy:  0.428
313/313 [==============================] - 1s 3